In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns',False)

delivery = pd.read_csv('../Data/IPL_Ball_by_Ball_2008_2022.csv')
match = pd.read_csv('../Data/IPL_Matches_2008_2022.csv')

In [2]:
match.head()

,ID,City,Date,Season,MatchNumber,Team1,Team2,Venue,TossWinner,TossDecision,SuperOver,WinningTeam,WonBy,Margin,method,Player_of_Match,Team1Players,Team2Players,Umpire1,Umpire2
0,1312200,Ahmedabad,2022-05-29,2022,Final,Rajasthan Royals,Gujarat Titans,"Narendra Modi Stadium, Ahmedabad",Rajasthan Royals,bat,N,Gujarat Titans,Wickets,7.0,NaN,HH Pandya,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",CB Gaffaney,Nitin Menon
1,1312199,Ahmedabad,2022-05-27,2022,Qualifier 2,Royal Challengers Bangalore,Rajasthan Royals,"Narendra Modi Stadium, Ahmedabad",Rajasthan Royals,field,N,Rajasthan Royals,Wickets,7.0,NaN,JC Buttler,"['V Kohli', 'F du Plessis', 'RM Patidar', 'GJ ...","['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...",CB Gaffaney,Nitin Menon
2,1312198,Kolkata,2022-05-25,2022,Eliminator,Royal Challengers Bangalore,Lucknow Super Giants,"Eden Gardens, Kolkata",Lucknow Super Giants,field,N,Royal Challengers Bangalore,Runs,14.0,NaN,RM Patidar,"['V Kohli', 'F du Plessis', 'RM Patidar', 'GJ ...","['Q de Kock', 'KL Rahul', 'M Vohra', 'DJ Hooda...",J Madanagopal,MA Gough
3,1312197,Kolkata,2022-05-24,2022,Qualifier 1,Rajasthan Royals,Gujarat Titans,"Eden Gardens, Kolkata",Gujarat Titans,field,N,Gujarat Titans,Wickets,7.0,NaN,DA Miller,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",BNJ Oxenford,VK Sharma
4,1304116,Mumbai,2022-05-22,2022,70,Sunrisers Hyderabad,Punjab Kings,"Wankhede Stadium, Mumbai",Sunrisers Hyderabad,bat,N,Punjab Kings,Wickets,5.0,NaN,Harpreet Brar,"['PK Garg', 'Abhishek Sharma', 'RA Tripathi', ...","['JM Bairstow', 'S Dhawan', 'M Shahrukh Khan',...",AK Chaudhary,NA Patwardhan


In [3]:
delivery.head()

,ID,innings,overs,ballnumber,batter,bowler,non-striker,extra_type,batsman_run,extras_run,total_run,non_boundary,isWicketDelivery,player_out,kind,fielders_involved,BattingTeam
0,1312200,1,0,1,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals
1,1312200,1,0,2,YBK Jaiswal,Mohammed Shami,JC Buttler,legbyes,0,1,1,0,0,NaN,NaN,NaN,Rajasthan Royals
2,1312200,1,0,3,JC Buttler,Mohammed Shami,YBK Jaiswal,NaN,1,0,1,0,0,NaN,NaN,NaN,Rajasthan Royals
3,1312200,1,0,4,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals
4,1312200,1,0,5,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals


In [4]:
total_score_df =delivery.groupby(['ID','innings']).sum()['total_run'].reset_index()

In [5]:
total_score_df = total_score_df[total_score_df['innings']==1]

In [6]:
total_score_df 

,ID,innings,total_run
0,335982,1,222
2,335983,1,240
4,335984,1,129
6,335985,1,165
8,335986,1,110
...,...,...,...
1918,1304116,1,157
1920,1312197,1,188
1922,1312198,1,207
1924,1312199,1,157


In [7]:
match_df =match.merge(total_score_df[['ID','total_run']],left_on='ID',right_on='ID')

In [8]:
match_df.head()

,ID,City,Date,Season,MatchNumber,Team1,Team2,Venue,TossWinner,TossDecision,SuperOver,WinningTeam,WonBy,Margin,method,Player_of_Match,Team1Players,Team2Players,Umpire1,Umpire2,total_run
0,1312200,Ahmedabad,2022-05-29,2022,Final,Rajasthan Royals,Gujarat Titans,"Narendra Modi Stadium, Ahmedabad",Rajasthan Royals,bat,N,Gujarat Titans,Wickets,7.0,NaN,HH Pandya,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",CB Gaffaney,Nitin Menon,130
1,1312199,Ahmedabad,2022-05-27,2022,Qualifier 2,Royal Challengers Bangalore,Rajasthan Royals,"Narendra Modi Stadium, Ahmedabad",Rajasthan Royals,field,N,Rajasthan Royals,Wickets,7.0,NaN,JC Buttler,"['V Kohli', 'F du Plessis', 'RM Patidar', 'GJ ...","['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...",CB Gaffaney,Nitin Menon,157
2,1312198,Kolkata,2022-05-25,2022,Eliminator,Royal Challengers Bangalore,Lucknow Super Giants,"Eden Gardens, Kolkata",Lucknow Super Giants,field,N,Royal Challengers Bangalore,Runs,14.0,NaN,RM Patidar,"['V Kohli', 'F du Plessis', 'RM Patidar', 'GJ ...","['Q de Kock', 'KL Rahul', 'M Vohra', 'DJ Hooda...",J Madanagopal,MA Gough,207
3,1312197,Kolkata,2022-05-24,2022,Qualifier 1,Rajasthan Royals,Gujarat Titans,"Eden Gardens, Kolkata",Gujarat Titans,field,N,Gujarat Titans,Wickets,7.0,NaN,DA Miller,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",BNJ Oxenford,VK Sharma,188
4,1304116,Mumbai,2022-05-22,2022,70,Sunrisers Hyderabad,Punjab Kings,"Wankhede Stadium, Mumbai",Sunrisers Hyderabad,bat,N,Punjab Kings,Wickets,5.0,NaN,Harpreet Brar,"['PK Garg', 'Abhishek Sharma', 'RA Tripathi', ...","['JM Bairstow', 'S Dhawan', 'M Shahrukh Khan',...",AK Chaudhary,NA Patwardhan,157


In [9]:
match_df = match_df[~(match_df['method']=='D/L')]

In [10]:
match_df = match_df[['ID','Venue','WinningTeam','total_run','Team1','Team2']]

In [11]:
delivery_df = match_df.merge(delivery,on='ID')

In [12]:
delivery_df = delivery_df[delivery_df['innings']==2]

In [13]:
delivery_df.head()

,ID,Venue,WinningTeam,total_run_x,Team1,Team2,innings,overs,ballnumber,batter,bowler,non-striker,extra_type,batsman_run,extras_run,total_run_y,non_boundary,isWicketDelivery,player_out,kind,fielders_involved,BattingTeam
120,1312200,"Narendra Modi Stadium, Ahmedabad",Gujarat Titans,130,Rajasthan Royals,Gujarat Titans,2,0,1,WP Saha,TA Boult,Shubman Gill,NaN,0,0,0,0,0,NaN,NaN,NaN,Gujarat Titans
121,1312200,"Narendra Modi Stadium, Ahmedabad",Gujarat Titans,130,Rajasthan Royals,Gujarat Titans,2,0,2,WP Saha,TA Boult,Shubman Gill,NaN,0,0,0,0,0,NaN,NaN,NaN,Gujarat Titans
122,1312200,"Narendra Modi Stadium, Ahmedabad",Gujarat Titans,130,Rajasthan Royals,Gujarat Titans,2,0,3,WP Saha,TA Boult,Shubman Gill,NaN,1,0,1,0,0,NaN,NaN,NaN,Gujarat Titans
123,1312200,"Narendra Modi Stadium, Ahmedabad",Gujarat Titans,130,Rajasthan Royals,Gujarat Titans,2,0,4,Shubman Gill,TA Boult,WP Saha,NaN,0,0,0,0,0,NaN,NaN,NaN,Gujarat Titans
124,1312200,"Narendra Modi Stadium, Ahmedabad",Gujarat Titans,130,Rajasthan Royals,Gujarat Titans,2,0,5,Shubman Gill,TA Boult,WP Saha,NaN,0,0,0,0,0,NaN,NaN,NaN,Gujarat Titans


In [14]:
delivery_df['current_score']=delivery_df.groupby('ID')['total_run_y'].cumsum()

In [15]:
delivery_df['runs_left'] = delivery_df['total_run_x']-delivery_df['current_score']+1

In [16]:
delivery_df['balls_left']=120-(delivery_df['overs']*6+delivery_df['ballnumber'])

In [17]:
delivery_df.head()

,ID,Venue,WinningTeam,total_run_x,Team1,Team2,innings,overs,ballnumber,batter,bowler,non-striker,extra_type,batsman_run,extras_run,total_run_y,non_boundary,isWicketDelivery,player_out,kind,fielders_involved,BattingTeam,current_score,runs_left,balls_left
120,1312200,"Narendra Modi Stadium, Ahmedabad",Gujarat Titans,130,Rajasthan Royals,Gujarat Titans,2,0,1,WP Saha,TA Boult,Shubman Gill,NaN,0,0,0,0,0,NaN,NaN,NaN,Gujarat Titans,0,131,119
121,1312200,"Narendra Modi Stadium, Ahmedabad",Gujarat Titans,130,Rajasthan Royals,Gujarat Titans,2,0,2,WP Saha,TA Boult,Shubman Gill,NaN,0,0,0,0,0,NaN,NaN,NaN,Gujarat Titans,0,131,118
122,1312200,"Narendra Modi Stadium, Ahmedabad",Gujarat Titans,130,Rajasthan Royals,Gujarat Titans,2,0,3,WP Saha,TA Boult,Shubman Gill,NaN,1,0,1,0,0,NaN,NaN,NaN,Gujarat Titans,1,130,117
123,1312200,"Narendra Modi Stadium, Ahmedabad",Gujarat Titans,130,Rajasthan Royals,Gujarat Titans,2,0,4,Shubman Gill,TA Boult,WP Saha,NaN,0,0,0,0,0,NaN,NaN,NaN,Gujarat Titans,1,130,116
124,1312200,"Narendra Modi Stadium, Ahmedabad",Gujarat Titans,130,Rajasthan Royals,Gujarat Titans,2,0,5,Shubman Gill,TA Boult,WP Saha,NaN,0,0,0,0,0,NaN,NaN,NaN,Gujarat Titans,1,130,115


In [18]:
wickets = delivery_df.groupby('ID')['isWicketDelivery'].cumsum().values

In [19]:
delivery_df['wickets']=10- wickets

In [20]:
delivery_df.head()

,ID,Venue,WinningTeam,total_run_x,Team1,Team2,innings,overs,ballnumber,batter,bowler,non-striker,extra_type,batsman_run,extras_run,total_run_y,non_boundary,isWicketDelivery,player_out,kind,fielders_involved,BattingTeam,current_score,runs_left,balls_left,wickets
120,1312200,"Narendra Modi Stadium, Ahmedabad",Gujarat Titans,130,Rajasthan Royals,Gujarat Titans,2,0,1,WP Saha,TA Boult,Shubman Gill,NaN,0,0,0,0,0,NaN,NaN,NaN,Gujarat Titans,0,131,119,10
121,1312200,"Narendra Modi Stadium, Ahmedabad",Gujarat Titans,130,Rajasthan Royals,Gujarat Titans,2,0,2,WP Saha,TA Boult,Shubman Gill,NaN,0,0,0,0,0,NaN,NaN,NaN,Gujarat Titans,0,131,118,10
122,1312200,"Narendra Modi Stadium, Ahmedabad",Gujarat Titans,130,Rajasthan Royals,Gujarat Titans,2,0,3,WP Saha,TA Boult,Shubman Gill,NaN,1,0,1,0,0,NaN,NaN,NaN,Gujarat Titans,1,130,117,10
123,1312200,"Narendra Modi Stadium, Ahmedabad",Gujarat Titans,130,Rajasthan Royals,Gujarat Titans,2,0,4,Shubman Gill,TA Boult,WP Saha,NaN,0,0,0,0,0,NaN,NaN,NaN,Gujarat Titans,1,130,116,10
124,1312200,"Narendra Modi Stadium, Ahmedabad",Gujarat Titans,130,Rajasthan Royals,Gujarat Titans,2,0,5,Shubman Gill,TA Boult,WP Saha,NaN,0,0,0,0,0,NaN,NaN,NaN,Gujarat Titans,1,130,115,10


In [21]:
delivery_df['crr']=delivery_df.current_score*6/(120-delivery_df.balls_left)

In [22]:
delivery_df['rrr']=delivery_df.runs_left*6/delivery_df.balls_left

In [23]:
delivery_df['BowlingTeam'] = np.where(delivery_df.BattingTeam == delivery_df.Team1, delivery_df.Team2, delivery_df.Team1)

In [24]:
#delivery_df['winner'] = np.where(delivery_df.BattingTeam == delivery_df.WinningTeam, 1, 0)

In [25]:
delivery_df

,ID,Venue,WinningTeam,total_run_x,Team1,Team2,innings,overs,ballnumber,batter,bowler,non-striker,extra_type,batsman_run,extras_run,total_run_y,non_boundary,isWicketDelivery,player_out,kind,fielders_involved,BattingTeam,current_score,runs_left,balls_left,wickets,crr,rrr,BowlingTeam
120,1312200,"Narendra Modi Stadium, Ahmedabad",Gujarat Titans,130,Rajasthan Royals,Gujarat Titans,2,0,1,WP Saha,TA Boult,Shubman Gill,NaN,0,0,0,0,0,NaN,NaN,NaN,Gujarat Titans,0,131,119,10,0.000000,6.605042,Rajasthan Royals
121,1312200,"Narendra Modi Stadium, Ahmedabad",Gujarat Titans,130,Rajasthan Royals,Gujarat Titans,2,0,2,WP Saha,TA Boult,Shubman Gill,NaN,0,0,0,0,0,NaN,NaN,NaN,Gujarat Titans,0,131,118,10,0.000000,6.661017,Rajasthan Royals
122,1312200,"Narendra Modi Stadium, Ahmedabad",Gujarat Titans,130,Rajasthan Royals,Gujarat Titans,2,0,3,WP Saha,TA Boult,Shubman Gill,NaN,1,0,1,0,0,NaN,NaN,NaN,Gujarat Titans,1,130,117,10,2.000000,6.666667,Rajasthan Royals
123,1312200,"Narendra Modi Stadium, Ahmedabad",Gujarat Titans,130,Rajasthan Royals,Gujarat Titans,2,0,4,Shubman Gill,TA Boult,WP Saha,NaN,0,0,0,0,0,NaN,NaN,NaN,Gujarat Titans,1,130,116,10,1.500000,6.724138,Rajasthan Royals
124,1312200,"Narendra Modi Stadium, Ahmedabad",Gujarat Titans,130,Rajasthan Royals,Gujarat Titans,2,0,5,Shubman Gill,TA Boult,WP Saha,NaN,0,0,0,0,0,NaN,NaN,NaN,Gujarat Titans,1,130,115,10,1.200000,6.782609,Rajasthan Royals
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222741,335982,M Chinnaswamy Stadium,Kolkata Knight Riders,222,Royal Challengers Bangalore,Kolkata Knight Riders,2,14,5,P Kumar,I Sharma,SB Joshi,legbyes,0,1,1,0,0,NaN,NaN,NaN,Royal Challengers Bangalore,80,143,31,1,5.393258,27.677419,Kolkata Knight Riders
222742,335982,M Chinnaswamy Stadium,Kolkata Knight Riders,222,Royal Challengers Bangalore,Kolkata Knight Riders,2,14,6,SB Joshi,I Sharma,P Kumar,NaN,1,0,1,0,0,NaN,NaN,NaN,Royal Challengers Bangalore,81,142,30,1,5.400000,28.400000,Kolkata Knight Riders
222743,335982,M Chinnaswamy Stadium,Kolkata Knight Riders,222,Royal Challengers Bangalore,Kolkata Knight Riders,2,14,7,P Kumar,I Sharma,SB Joshi,NaN,0,0,0,0,0,NaN,NaN,NaN,Royal Challengers Bangalore,81,142,29,1,5.340659,29.379310,Kolkata Knight Riders
222744,335982,M Chinnaswamy Stadium,Kolkata Knight Riders,222,Royal Challengers Bangalore,Kolkata Knight Riders,2,15,1,SB Joshi,LR Shukla,P Kumar,wides,0,1,1,0,0,NaN,NaN,NaN,Royal Challengers Bangalore,82,141,29,1,5.406593,29.172414,Kolkata Knight Riders


In [26]:
def is_win(df):
    winner = []
    for item,row in df.iterrows():
        if row.WinningTeam == row.BattingTeam:
            winner.append(1)
        else:
            winner.append(0)

    return winner
            
delivery_df['winner']=is_win(delivery_df)

In [27]:
final_df =delivery_df[['ID','BattingTeam','BowlingTeam','Venue','runs_left','balls_left','wickets','total_run_x','crr','rrr','winner']]

In [28]:
final_df.head()

,ID,BattingTeam,BowlingTeam,Venue,runs_left,balls_left,wickets,total_run_x,crr,rrr,winner
120,1312200,Gujarat Titans,Rajasthan Royals,"Narendra Modi Stadium, Ahmedabad",131,119,10,130,0.0,6.605042,1
121,1312200,Gujarat Titans,Rajasthan Royals,"Narendra Modi Stadium, Ahmedabad",131,118,10,130,0.0,6.661017,1
122,1312200,Gujarat Titans,Rajasthan Royals,"Narendra Modi Stadium, Ahmedabad",130,117,10,130,2.0,6.666667,1
123,1312200,Gujarat Titans,Rajasthan Royals,"Narendra Modi Stadium, Ahmedabad",130,116,10,130,1.5,6.724138,1
124,1312200,Gujarat Titans,Rajasthan Royals,"Narendra Modi Stadium, Ahmedabad",130,115,10,130,1.2,6.782609,1


In [29]:
final_df =final_df[final_df.balls_left !=0]

In [30]:
final_df =final_df.sample(final_df.shape[0])

In [31]:
final_df.head()

,ID,BattingTeam,BowlingTeam,Venue,runs_left,balls_left,wickets,total_run_x,crr,rrr,winner
69108,1136587,Mumbai Indians,Chennai Super Kings,"Maharashtra Cricket Association Stadium, Pune",69,42,9,169,7.769231,9.857143,1
100797,980911,Gujarat Lions,Rising Pune Supergiant,Saurashtra Cricket Association Stadium,153,113,10,163,9.428571,8.123894,1
67871,1136593,Kolkata Knight Riders,Chennai Super Kings,"Eden Gardens, Kolkata",82,53,7,177,8.597015,9.283019,1
215406,336014,Deccan Chargers,Kolkata Knight Riders,"Rajiv Gandhi International Stadium, Uppal",107,43,5,204,7.636364,14.930233,0
174959,501228,Chennai Super Kings,Pune Warriors,"Dr DY Patil Sports Academy, Mumbai",128,100,10,141,4.200000,7.680000,1


In [32]:
final_df.to_csv('final.csv', index=False)

In [33]:
from sklearn.model_selection import train_test_split
X = final_df.drop(columns=['winner','ID'])
y = final_df['winner']

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1) 

In [34]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

In [35]:
ohe = OneHotEncoder()
ohe.fit_transform(X.iloc[:,:3])
category = ohe.categories_

preprocess = ColumnTransformer(transformers=[
    ('cat', OneHotEncoder(categories=category,sparse_output=False,drop='first'), ['BattingTeam', 'BowlingTeam', 'Venue'])
],remainder='passthrough')

In [36]:
logreg = LogisticRegression(solver='liblinear')
model = Pipeline(
    steps= [
        ('preprocess',preprocess),
        ('modle', logreg)
    ]
)

In [37]:
model.fit(X_train,y_train)

Pipeline(steps=[('preprocess',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat',
                                                  OneHotEncoder(categories=[array(['Chennai Super Kings', 'Deccan Chargers', 'Delhi Capitals',
       'Delhi Daredevils', 'Gujarat Lions', 'Gujarat Titans',
       'Kings XI Punjab', 'Kochi Tuskers Kerala', 'Kolkata Knight Riders',
       'Lucknow Super Giants', 'Mumbai Indians', 'Pune Warriors',
       'Punjab King...
       'Shaheed Veer Narayan Singh International Stadium',
       'Sharjah Cricket Stadium', "St George's Park",
       'Subrata Roy Sahara Stadium', 'SuperSport Park',
       'Vidarbha Cricket Association Stadium, Jamtha',
       'Wankhede Stadium, Mumbai', 'Zayed Cricket Stadium, Abu Dhabi'],
      dtype=object)],
                                                                drop='first',
                                                                sparse_output=False),
                                                  ['BattingTeam', 'BowlingTeam',
                                                   'Venue'])])),
                ('modle', LogisticRegression(solver='liblinear'))])

In [38]:
y_pred = model.predict_proba(X_test)

In [39]:
y_pred

array([[8.04100292e-01, 1.95899708e-01],
       [7.14391541e-02, 9.28560846e-01],
       [6.50368203e-01, 3.49631797e-01],
       ...,
       [9.99965581e-01, 3.44186619e-05],
       [9.99096443e-01, 9.03556580e-04],
       [9.71237557e-01, 2.87624430e-02]])

In [42]:
np.round(y_pred.T[0]*100,1)

array([ 80.4,   7.1,  65. , ..., 100. ,  99.9,  97.1])

In [43]:
np.round(y_pred.T[1]*100,1)

array([19.6, 92.9, 35. , ...,  0. ,  0.1,  2.9])

In [40]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 107380 entries, 69108 to 157291
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   ID           107380 non-null  int64  
 1   BattingTeam  107380 non-null  object 
 2   BowlingTeam  107380 non-null  object 
 3   Venue        107380 non-null  object 
 4   runs_left    107380 non-null  int64  
 5   balls_left   107380 non-null  int64  
 6   wickets      107380 non-null  int64  
 7   total_run_x  107380 non-null  int64  
 8   crr          107380 non-null  float64
 9   rrr          107380 non-null  float64
 10  winner       107380 non-null  int64  
dtypes: float64(2), int64(6), object(3)
memory usage: 9.8+ MB


In [41]:
X_test

,BattingTeam,BowlingTeam,Venue,runs_left,balls_left,wickets,total_run_x,crr,rrr
164135,Kolkata Knight Riders,Rajasthan Royals,Sawai Mansingh Stadium,36,13,3,164,7.233645,16.615385
34661,Sunrisers Hyderabad,Royal Challengers Bangalore,Sharjah Cricket Stadium,47,60,8,120,7.400000,4.700000
142623,Pune Warriors,Sunrisers Hyderabad,"Maharashtra Cricket Association Stadium, Pune",25,24,6,119,5.937500,6.250000
168832,Delhi Daredevils,Punjab Kings,Himachal Pradesh Cricket Association Stadium,35,4,2,170,7.034483,52.500000
97253,Kings XI Punjab,Punjab Kings,"Punjab Cricket Association IS Bindra Stadium, ...",153,87,8,189,6.727273,10.551724
...,...,...,...,...,...,...,...,...,...
191028,Kings XI Punjab,Punjab Kings,"Punjab Cricket Association IS Bindra Stadium, ...",165,98,9,183,5.181818,10.102041
99729,Delhi Daredevils,Royal Challengers Bangalore,M Chinnaswamy Stadium,8,11,8,191,10.128440,4.363636
137087,Kings XI Punjab,Chennai Super Kings,"MA Chidambaram Stadium, Chepauk, Chennai",53,23,7,186,8.288660,13.826087
140153,Kings XI Punjab,Delhi Capitals,"Arun Jaitley Stadium, Delhi",115,115,9,120,7.200000,6.000000


In [44]:
import joblib

joblib.dump(model,'log_reg.pkl')

['log_reg.pkl']